In [1]:
import os
from scripts.MicroBrix import MicroBrix
import geopandas as gpd
import json
import requests

In [2]:
def get_data():
    import matplotlib.pyplot as plt
    layers = []
    layer_id = 'id_de_la_capa'
    layer_type = 'geojsonbase'
    layer = gpd.read_file("./data/grid_combined/grid_combined.shp")

    # Parametros para el color del plot
    colormap_name = 'jet' # buscar color maps en matplotlib
    colormap = plt.get_cmap(colormap_name)
    value_column = 'LEGOS'
    min_value = 0 # valor minimo de la escala
    max_value = 100 # valor maximo de la escala
    if value_column is None:
        raise ValueError("Value column must be set before calculating colors.")
    norm = plt.Normalize(vmin=min_value, vmax=max_value)
    layer['color'] = layer[value_column].apply(lambda x: colormap(norm(x))[:3])
    layer['color'] = layer['color'].apply(lambda rgb: tuple(int(c * 255) for c in rgb))
    layer['fill'] = layer['color'].apply(lambda x: '#%02x%02x%02x' % (x[0], x[1], x[2]))

    features = json.loads(layer.to_json())
    layers.append({"id": layer_id, "type": layer_type, "data": features, "properties": {"filled": True}})

    return {
        "layers": layers,
        "numeric": []
    }

def get_data_from_server():
   dataset_server_address = "http://indicator-api:9875"
   response = requests.get(f'{dataset_server_address}/get_layers_and_numeric/')
   data = response.json()
   print(data)
   return data["layers"], data["numeric"]
   # return {
   #     "layers": data['layers'],
   #     "numeric": data['numeric']
   # }

def serve_indicators(geogrid, geogrid_data):
    return get_data_from_server()

In [ ]:
connection = MicroBrix(
    table_name='costanera_conce',
    module_function= serve_indicators,
    keep_updating=True,
    quietly=True,
)
connection.listen()

## Opened connection


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:
connection.stop()

In [5]:
connection.ws.close()